In [1]:
import pandas as pd
from utils import load_filtered_data

In [2]:
city = 'Sicklerville'
name = 'categories'

# load city data
data = load_filtered_data(city)

In [3]:
# creating the nodes

nodes = data['user'][['user_id', 'name']]
nodes = nodes.rename(columns={'user_id': 'id', 'name': 'people_name'})

nodes

,id,people_name
0,RgDVC3ZUBqpEe6Y1kPhIpw,Monica
1,zkamNMEjihh3zN7lC7_WVw,Matthew
2,gVFxZMcuG_Tal2_TnpmUPg,Stephanie
3,g0_x4kVvJAYuk96oCcbOmw,Michael
4,-NbeVN5tnwdyYAvdNkKMjw,Dominic
...,...,...
3073,CUE13op55l9WZp6fBTUWnA,Nicole
3074,cACn6Qc1owO7hV_ZhcP74Q,Allison
3075,SpTYEhfInvALG1le1uu21w,Ray
3076,sGOCaCP7_SMlaRKPNO__CQ,Tahjae


In [4]:
temp_edges_struct = {'from': [], 'to': [], 'weight': [], 'type': []}

# Splitting categories into individual entries
categories_expanded = data['business']['categories'].str.split(', ').explode().dropna().unique()

for category in categories_expanded:
    businesses_in_category = data['business'][data['business']['categories'].apply(lambda x: category in x)]
    users_who_reviewed_category = data['review'][data['review']['business_id'].isin(businesses_in_category['business_id'])]['user_id'].unique()
    for i in range(len(users_who_reviewed_category)-1):
        for j in range(i+1, len(users_who_reviewed_category)):
            # from i to j
            temp_edges_struct['from'].append(users_who_reviewed_category[i])
            temp_edges_struct['to'].append(users_who_reviewed_category[j])
            temp_edges_struct['weight'].append(1)
            temp_edges_struct['type'].append('reviewed-same-category')
            # from j to i
            temp_edges_struct['from'].append(users_who_reviewed_category[j])
            temp_edges_struct['to'].append(users_who_reviewed_category[i])
            temp_edges_struct['weight'].append(1)
            temp_edges_struct['type'].append('reviewed-same-category')

edges = pd.DataFrame(temp_edges_struct)
edges = edges.drop_duplicates()

edges

,from,to,weight,type
0,iJe_4Z_731GUnEFYZ8xREw,8DGc1UEegk6SBGV39OJtQA,1,reviewed-same-category
1,8DGc1UEegk6SBGV39OJtQA,iJe_4Z_731GUnEFYZ8xREw,1,reviewed-same-category
2,iJe_4Z_731GUnEFYZ8xREw,1-icdXJv2v9MMbytcVLBcQ,1,reviewed-same-category
3,1-icdXJv2v9MMbytcVLBcQ,iJe_4Z_731GUnEFYZ8xREw,1,reviewed-same-category
4,iJe_4Z_731GUnEFYZ8xREw,FV6QtuA5P52Cm5q7SOeMLQ,1,reviewed-same-category
...,...,...,...,...
8191423,o8xoVrWWrDiJX7H12cR9Fg,E16JxbBp6MEfbSKIMyA0Hw,1,reviewed-same-category
8191426,E16JxbBp6MEfbSKIMyA0Hw,7xWrRJ5kfgcQyQ3UL3fImQ,1,reviewed-same-category
8191427,7xWrRJ5kfgcQyQ3UL3fImQ,E16JxbBp6MEfbSKIMyA0Hw,1,reviewed-same-category
8191428,E16JxbBp6MEfbSKIMyA0Hw,W_vejv-B63JOZHUN_2Zrtg,1,reviewed-same-category


In [5]:
# save data
nodes.to_csv(f'nodes_and_edges/{city}_{name}_nodes.csv', index=False)
edges.to_csv(f'nodes_and_edges/{city}_{name}_edges.csv', index=False)